In [1]:
from erddapy import ERDDAP
import pandas as pd
import requests
import numpy as np
import xarray as xr

In [2]:
server = 'http://erddap.maracoos.org/erddap'
protocol = 'tabledap'

In [3]:
e = ERDDAP(
    server=server,
    protocol=protocol)

In [4]:
df = pd.read_csv(e.get_search_url(response="csv", search_for="vims"))

In [5]:
df.to_csv('ERDDAP_ChesdBay_datasets.csv')

In [6]:
df.keys()

Index(['griddap', 'Subset', 'tabledap', 'Make A Graph', 'wms', 'files',
       'Title', 'Summary', 'FGDC', 'ISO 19115', 'Info', 'Background Info',
       'RSS', 'Email', 'Institution', 'Dataset ID'],
      dtype='object')

In [7]:
dataset_list = []
for subset in df['Subset']:
    dataset_list.append(subset.strip('.subset'))

    # add a header before saving it to a file
dataset_list = ['ChesB']+dataset_list

filename= dataset_list[1].split('/')[-1]+'.txt'
filename

'cbp_CB2_2.txt'

In [8]:
# Open a file
fo = open('ChesBay_dataset_08-13-2020.csv', "w")
# Write sequence of lines at the end of the file.

line = fo.write(',\n'.join( dataset_list ))
# Close opend file
fo.close()

In [9]:
e.get_info_url(dataset_id=df['Dataset ID'][0], response='nc')

'http://erddap.maracoos.org/erddap/info/cbp_CB2_2/index.nc'

In [11]:
e.get_download_url(dataset_id=df['Dataset ID'][0])

'http://erddap.maracoos.org/erddap/tabledap/cbp_CB2_2.html?'

In [12]:
e.dataset_id = df['Dataset ID'][93]
ds = e.to_xarray(decode_times=False)
ds.dims

Frozen(SortedKeysDict({'row': 181}))

In [ ]:
def get_geophysical_variables(ds):
    """
    Returns a list of variable names for the variables detected as geophysical
    variables.
    :param netCDF4.Dataset nc: An open netCDF dataset
    """
    parameters = []
    for variable in ds.variables:
        if is_geophysical(ds, variable):
            parameters.append(variable)
    return parameters

In [36]:
ncvar = ds['SALINITY'] #dir(ds.variables)
# print(ncvar)



In [40]:
getattr(ncvar, "standard_name")

'sea_water_practical_salinity'

In [ ]:
if getattr(ncvar, "Conventions", None):
    print(False)

# Check for axis
if getattr(ncvar, "axis", None):
    print(False)

In [ ]:
standard_name = getattr(ncvar, "standard_name", "")
if standard_name in {
        "time",
        "latitude",
        "longitude",
        "height",
        "depth",
        "altitude",
    }:
        print(False)

In [ ]:
parameters = get_geophysical_variables(ds)

In [ ]:
for dimension in ds.dims:
    print(dimension)
    if dimension in ds.variables:
        print(dimension)
        if ds.variables[dimension].dims == (dimension,):
            print(dimension)
    else:
        print('no match')

In [ ]:
dim_list = []
for variable in ds.variables:
    dim_list.append(ds.variables[variable].dims)
print(np.unique(dim_list))

In [ ]:
val_str = []
for data_id in df['Dataset ID']:
    e.dataset_id = data_id
    ds = e.to_xarray(decode_times=False)
    try:
        val_str.append(ds.attrs['creator_insitution']) #'Conventions'
    except KeyError:
        try:
            print(data_id, 'no creator_institution')
            val_str.append(ds.attrs['institution'])
        except KeyError:
            print(data_id, 'no institution')

In [ ]:
print(len(val_str))
print(np.unique(val_str), len(np.unique(val_str)))

In [ ]:
for url in dataset_list[92:95]:
    option = 'lenient'
    filename= url.split('/')[-1]+'.txt'
    dirname='compliance_checker_'+ option
    !compliance-checker --test=acdd:1.3 --format=text --output=./$dirname/accd_report/$filename $url -c $option -v
    !compliance-checker --test=cf:1.6 --format=text --output=./$dirname/cf_report/$filename $url -c $option -v
   

In [ ]:
!compliance-checker --test=cf:1.6 --format=text --output=./cf_report/test.txt "http://erddap.maracoos.org/erddap/tabledap/cbp_CB2_2" -c lenient -v

In [ ]:
df_1=xr.open_dataset('/Users/leilabelabassi/Downloads/sp029-20200722T2029_627c_21a2_a3eb.nc')

In [ ]:
df_1.attrs['Conventions']

In [ ]:
!compliance-checker --test acdd:1.3 "http://erddap.maracoos.org/erddap/tabledap/cbp_CB2_2" -c lenient -v

In [ ]:
!compliance-checker --test ioos:1.2 "http://erddap.maracoos.org/erddap/tabledap/cbp_CB2_2" -c lenient -v